In [ ]:
#@title Select your network details

import numpy as np
import pandas as pd
import json
import textwrap

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import IPython.display
from IPython.display import display, clear_output
from urllib.parse import urlencode
from urllib.parse import quote

import plotly.graph_objects as go

def dropdown_menu_widget_healthy_aging():
    
    output = widgets.Output()

    dropdown_group = widgets.Dropdown(options = ['Bibliographic Coupling', 'Citation', 'Co-authorship', 'Co-citation'], value=None, description='Network Group:')
    dropdown_sub_category = widgets.Dropdown(options = ['Authors','Countries', 'Documents', 'Sources'], value='Authors', description='Sub_category:')
    dropdown_year = widgets.Dropdown(options = ['Until_2019', 2020, 2021, 2022], 
                                               value=2020, description='Year:')
    
    global url

    url = "https://raw.githubusercontent.com/SisonkeBiotik-Africa/AfriBioML/main/Network/{group}%20-%20{val_sub_category}/{val_year}.json"

    def output_by_state(val_group, val_sub_category, val_year):
        group = quote(val_group)
        url1 = url.format(group=group, val_sub_category=val_sub_category, val_year=val_year)
        #f"https://raw.githubusercontent.com/SisonkeBiotik-Africa/AfriBioML/main/Network/{group}%20-%20{val_sub_category}/{str(val_year)}.json"
        # make the analysis
        def make_analysis (url_par):
            data = pd.read_json(url_par, encoding='utf-8-sig', orient='records')
            full_data = pd.DataFrame.from_dict(data.iloc[0]['network'])
            final_df = pd.concat([full_data, pd.json_normalize(full_data['weights'])], axis=1)
            if 'scores' in final_df:
                final_df = final_df.drop(['weights', 'scores'], axis=1)
            else:
                final_df = final_df.drop('weights', axis=1)
            return final_df
        
        
        try:
            df1 = make_analysis(url1)
            if 'Total link strength' in df1.columns:
                display(df1.sort_values('Total link strength', ascending=False)   .head(15))  
            else:
                display(df1  .head(6))  
            print(df1.info())

           
        except Exception as err:
            print("Location does not Exist")

            raise err

        # other 2 
        # print(url.format(group=group, val_sub_category=val_sub_category, val_year=2021))
        # df2 = make_analysis(url.format(group=group, val_sub_category=val_sub_category, val_year=2021))
        # if 'Total link strength' in df2.columns:
        #         display(df2.sort_values('Total link strength', ascending=False)   .head(6))  
        # else:
        #     display(df2  .head(6))
        # print(df2.head())
        # df3 = make_analysis(url.format(group=group, val_sub_category=val_sub_category, val_year=2022))
        # df3.head()


    def dropdown_group_eventhandler(change):
        """
        Eventhandler for the state dropdown widget
        """
        display(input_widgets)
        state_choice = change.new
        output_by_state(state_choice, dropdown_sub_category.value, dropdown_year.value)
        IPython.display.clear_output(wait=True)            

    def dropdown_sub_category_eventhandler(change):
        """
        Eventhandler for the question dropdown widget
        """
        display(input_widgets)
        question_choice = change.new
        output_by_state(dropdown_group.value, question_choice, dropdown_year.value)
        IPython.display.clear_output(wait=True)

    def dropdown_year_eventhandler(change):     
        """
        Event handler for the year dropdown widget
        """
        display(input_widgets)
        stratification_choice = change.new
        output_by_state(dropdown_group.value, dropdown_sub_category.value, stratification_choice)
        IPython.display.clear_output(wait=True)
            
    dropdown_group.observe(dropdown_group_eventhandler, names='value')
    dropdown_sub_category.observe(dropdown_sub_category_eventhandler, names='value')
    dropdown_year.observe(dropdown_year_eventhandler, names='value')

    
    input_widgets = widgets.HBox([dropdown_group, dropdown_sub_category, dropdown_year])
    
    display(input_widgets)
    IPython.display.clear_output(wait=True)  

dropdown_menu_widget_healthy_aging()